In [1]:
import pandas as pd

## Small Dataset

In [ ]:
df = pd.read_csv('/Users/huyilun/Desktop/amazon_reviews_us_Music_v1_00.tsv', sep='\t', error_bad_lines=False)
a = df[:1]
#a.to_csv('/Users/huyilun/Desktop/20000.tsv', sep="\t")

## Total Number

In [2]:
def convert_format(file):
    df = pd.read_csv(file, sep='\t', error_bad_lines=False)
    return df

def total_numbers(dataframe):
    review_id = dataframe["review_id"]
    customer_id = dataframe["customer_id"]
    product_id = dataframe["product_id"]
    r_count = review_id.drop_duplicates().count()
    c_count = customer_id.drop_duplicates().count()
    p_count = product_id.drop_duplicates().count()
    return r_count, c_count, p_count

In [3]:
%%time
df = convert_format('/Users/huyilun/Desktop/20000.tsv')
r, c, p = total_numbers(df)
print ("Total number of reviews:", r)
print ("Total number of customers:", c)
print ("Total number of products:", p)

Total number of reviews: 20000
Total number of customers: 12076
Total number of products: 15567
CPU times: user 132 ms, sys: 23.7 ms, total: 156 ms
Wall time: 162 ms


## User_Reviews & Product_Reviews

In [4]:
import findspark
findspark.init()
from pyspark import SparkContext
import argparse

In [5]:
sc = SparkContext(appName="Stage One: Overall Statistics")
parser = argparse.ArgumentParser()
AllData = sc.textFile("/Users/huyilun/Desktop/20000.tsv")
data1 = AllData.filter(lambda x: 'customer_id' not in x)

In [6]:
def user_reviews(file):
    parts = file.split("\t")
    customer_id = parts[2]
    review_id = parts[3]
    return (customer_id), (review_id)

def product_reviews(file):
    parts = file.split("\t")
    product_id = parts[4]
    review_id = parts[3]
    return (product_id), (review_id)

def apartData(line):
    target = line[0]
    num_reviews = len(line[1])
    return (target, num_reviews)

def reviews_extract(line):
    total_review = line[1]
    return (total_review)

def median(data):
    review_list = data.collect()
    total = len(review_list)
    if total%2 > 0:
        return review_list[int(total/2) + 1]
    else:
        return (review_list[int(total/2)] + review_list[int(total/2) + 1]) / 2

In [7]:
data2 = data1.map(user_reviews)
data3 = data2.groupByKey().mapValues(tuple)
data4 = data3.map(apartData)
data5 = data4.sortBy(lambda x: (1 - x[1]))
lagest_review_user = data5.take(1)[0][1] # Answer 2.1
ten_top_reviews_users = data5.take(10) # Answer 2.2
data6 = data5.map(reviews_extract)
user_median_review = median(data6) # Answer 2.3

In [ ]:
data7 = data1.map(product_reviews)
data8 = data7.groupByKey().mapValues(tuple)
data9 = data8.map(apartData)
data10 = data9.sortBy(lambda x: (1 - x[1]))
lagest_review_product = data10.take(1)[0][1] # Answer 3.1
ten_top_reviews_products = data10.take(10) # Answer 3.2
data11 = data10.map(reviews_extract)
product_median_review = median(data11) # Answer 3.3

In [11]:
data3.count()

12145

In [ ]:
lagest_review_user

In [ ]:
ten_top_reviews_users

In [ ]:
user_median_review

In [ ]:
lagest_review_product

In [ ]:
ten_top_reviews_products

In [ ]:
product_median_review